In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("/kaggle/input/churn-in-telecoms-dataset/bigml_59c28831336c6604c800002a.csv")

In [ ]:
df.describe()

Задача классификации потому, что целевая переменная chunk - категориальная


In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
sns.catplot(x="churn", kind="count", palette="ch:.25", data=df)

Классы не збалансированы


In [ ]:
df["area code"].value_counts()

In [ ]:
df["state"].value_counts()

Штатов слишком много, для такого количества данных и они не вносят явного вклада в таргет (дальше будет видно). Номера телефоно вообще не нужны.


In [ ]:
df_1 = df.drop(["state","phone number"], axis = 1)

In [ ]:
y = df_1["churn"].map({False:0,True:1})

In [ ]:
df_2 = df_1.drop(["area code", "churn"], axis = 1)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix


In [ ]:
df_2["international plan"] =df_2["international plan"].map({"no":0, "yes":1})

In [ ]:
df_2["voice mail plan"] =df_2["voice mail plan"].map({"no":0, "yes":1})

In [ ]:
X = df_2
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("F1 score: ",f1_score(y_pred, y_test))

Скор значительно учше чем был c KNN

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [ ]:
import numpy as np
params = {"max_depth":np.arange(2,21)}
from sklearn.model_selection import GridSearchCV
tree_grid_depth = GridSearchCV(tree, params, cv=kf, scoring='f1')
tree_grid_depth.fit(X, y)
tree_grid_depth.best_params_

In [ ]:
params = {"min_samples_split":np.arange(2,21)}
tree = DecisionTreeClassifier(max_depth = 6)
tree_grid_samples_split = GridSearchCV(tree, params, cv=kf, scoring='f1')
tree_grid_samples_split.fit(X, y)
tree_grid_samples_split.best_params_

In [ ]:
params = {"min_samples_leaf":np.arange(2,21)}
tree = DecisionTreeClassifier(max_depth = 6, min_samples_split = 11)
tree_grid_samples_leaf = GridSearchCV(tree, params, cv=kf, scoring='f1')
tree_grid_samples_leaf.fit(X, y)
tree_grid_samples_leaf.best_params_

In [ ]:
X.shape

In [ ]:
params = {"max_features":np.arange(2,18)}
tree = DecisionTreeClassifier(max_depth = 6, min_samples_split = 11, min_samples_leaf = 3)
tree_grid_features = GridSearchCV(tree, params, cv=kf, scoring='f1')
tree_grid_features.fit(X, y)
tree_grid_features.best_params_

In [ ]:
results_depth = pd.DataFrame(tree_grid_depth.cv_results_)
result_samples_split = pd.DataFrame(tree_grid_samples_split.cv_results_)
result_samples_leaf = pd.DataFrame(tree_grid_samples_leaf.cv_results_)
result_features = pd.DataFrame(tree_grid_features.cv_results_)

In [ ]:
import matplotlib.pyplot as plt
fig, ((ax11,ax12),(ax21,ax22)) = plt.subplots(2,2, figsize = (10,10))

ax11.set_xlabel("max_depth")
ax11.set_ylabel("f1 score")
ax11.plot(results_depth["param_max_depth"], results_depth["mean_test_score"])

ax12.set_xlabel("min_samples_split")
ax12.plot(result_samples_split["param_min_samples_split"], result_samples_split["mean_test_score"])

ax21.set_xlabel("min_samples_leaf")
ax21.set_ylabel("f1 score")
ax21.plot(result_samples_leaf["param_min_samples_leaf"], result_samples_leaf["mean_test_score"])

ax22.set_xlabel("max_features")
ax22.plot(result_features["param_max_features"], result_features["mean_test_score"])

Вывод: один из хороших сетов параметров модели это: 
1. max_depth = 6
2. min_samples_split = 11 
3. min_samples_leaf = 3
4. max_features = 13

In [ ]:
best_tree = DecisionTreeClassifier(max_depth = 6, min_samples_split = 11, min_samples_leaf = 3, max_features = 13)
best_tree.fit(X, y)

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(best_tree, out_file='tree.dot', feature_names=X.columns)
print(open('tree.dot').read()) 

![](data:image/svg+xml;charset=utf-8,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%3C!DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%3C!--%20Generated%20by%20graphviz%20version%202.40.1%20(20161225.0304)%0A%20--%3E%3C!--%20Title%3A%20Tree%20Pages%3A%201%20--%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20width%3D%22993pt%22%20height%3D%22400pt%22%20viewBox%3D%220.00%200.00%20993.04%20400.00%22%3E%0A%3Cg%20id%3D%22graph0%22%20class%3D%22graph%22%20transform%3D%22scale(1%201)%20rotate(0)%20translate(4%20396)%22%3E%0A%3Ctitle%3ETree%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22%23ffffff%22%20stroke%3D%22transparent%22%20points%3D%22-4%2C4%20-4%2C-396%20989.0356%2C-396%20989.0356%2C4%20-4%2C4%22%2F%3E%0A%3C!--%200%20--%3E%0A%3Cg%20id%3D%22node1%22%20class%3D%22node%22%3E%0A%3Ctitle%3E0%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22566.9627%2C-392.1003%20407.5729%2C-392.1003%20407.5729%2C-316.6997%20566.9627%2C-316.6997%20566.9627%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22487.2678%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Etotal%20day%20charge%20%26lt%3B%3D%2044.96%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22487.2678%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.248%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22487.2678%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203333%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22487.2678%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2850%2C%20483%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%20--%3E%0A%3Cg%20id%3D%22node2%22%20class%3D%22node%22%3E%0A%3Ctitle%3E1%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22474.1672%2C-280.9003%20294.3684%2C-280.9003%20294.3684%2C-205.4997%20474.1672%2C-205.4997%20474.1672%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Ecustomer%20service%20calls%20%26lt%3B%3D%203.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.202%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%203122%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2766%2C%20356%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B1%20--%3E%0A%3Cg%20id%3D%22edge1%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B1%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M452.3967%2C-316.7528C443.9233%2C-307.6048%20434.7772%2C-297.7305%20426.0212%2C-288.2774%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22428.4662%2C-285.7665%20419.103%2C-280.8085%20423.3307%2C-290.5233%20428.4662%2C-285.7665%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22418.1466%22%20y%3D%22-301.5902%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3ETrue%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%20--%3E%0A%3Cg%20id%3D%22node9%22%20class%3D%22node%22%3E%0A%3Ctitle%3E8%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22715.5775%2C-280.9003%20524.9581%2C-280.9003%20524.9581%2C-205.4997%20715.5775%2C-205.4997%20715.5775%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Enumber%20vmail%20messages%20%26lt%3B%3D%206.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.479%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20211%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B84%2C%20127%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B8%20--%3E%0A%3Cg%20id%3D%22edge8%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B8%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M532.2955%2C-316.7528C543.5619%2C-307.3331%20555.7493%2C-297.1433%20567.3594%2C-287.4362%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22569.8596%2C-289.908%20575.2864%2C-280.8085%20565.3696%2C-284.5377%20569.8596%2C-289.908%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22573.0635%22%20y%3D%22-301.5095%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EFalse%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%20--%3E%0A%3Cg%20id%3D%22node3%22%20class%3D%22node%22%3E%0A%3Ctitle%3E2%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22281.42%2C-169.7003%20127.1156%2C-169.7003%20127.1156%2C-94.2997%20281.42%2C-94.2997%20281.42%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22204.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Einternational%20plan%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22204.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.147%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22204.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202871%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22204.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2642%2C%20229%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B2%20--%3E%0A%3Cg%20id%3D%22edge2%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B2%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M323.328%2C-205.5528C307.4939%2C-195.7708%20290.3155%2C-185.1583%20274.0638%2C-175.1184%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22275.4919%2C-171.8866%20265.1448%2C-169.6085%20271.8128%2C-177.8419%20275.4919%2C-171.8866%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%205%20--%3E%0A%3Cg%20id%3D%22node6%22%20class%3D%22node%22%3E%0A%3Ctitle%3E5%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22467.4892%2C-169.7003%20301.0464%2C-169.7003%20301.0464%2C-94.2997%20467.4892%2C-94.2997%20467.4892%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Etotal%20day%20minutes%20%26lt%3B%3D%20160.2%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20251%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22384.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B124%2C%20127%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B5%20--%3E%0A%3Cg%20id%3D%22edge5%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B5%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M384.2678%2C-205.5528C384.2678%2C-197.22%20384.2678%2C-188.2845%20384.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22387.7679%2C-179.6084%20384.2678%2C-169.6085%20380.7679%2C-179.6085%20387.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%203%20--%3E%0A%3Cg%20id%3D%22node4%22%20class%3D%22node%22%3E%0A%3Ctitle%3E3%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22126.3042%2C-58.6014%20.2314%2C-58.6014%20.2314%2C.2014%20126.3042%2C.2014%20126.3042%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.093%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%202604%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2263.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B2476%2C%20128%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B3%20--%3E%0A%3Cg%20id%3D%22edge3%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B3%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M152.5652%2C-94.3048C139.2837%2C-84.6215%20125.0419%2C-74.2382%20111.9206%2C-64.6717%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22113.6235%2C-61.5818%20103.4811%2C-58.5186%20109.4996%2C-67.2381%20113.6235%2C-61.5818%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%204%20--%3E%0A%3Cg%20id%3D%22node5%22%20class%3D%22node%22%3E%0A%3Ctitle%3E4%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22263.8043%2C-58.6014%20144.7313%2C-58.6014%20144.7313%2C.2014%20263.8043%2C.2014%20263.8043%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22204.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.47%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22204.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20267%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22204.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B166%2C%20101%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B4%20--%3E%0A%3Cg%20id%3D%22edge4%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B4%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M204.2678%2C-94.3048C204.2678%2C-85.9126%20204.2678%2C-76.9946%20204.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22207.7679%2C-68.5186%20204.2678%2C-58.5186%20200.7679%2C-68.5187%20207.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%206%20--%3E%0A%3Cg%20id%3D%22node7%22%20class%3D%22node%22%3E%0A%3Ctitle%3E6%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22387.8044%2C-58.6014%20282.7312%2C-58.6014%20282.7312%2C.2014%20387.8044%2C.2014%20387.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22335.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.222%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22335.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20102%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22335.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B13%2C%2089%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B6%20--%3E%0A%3Cg%20id%3D%22edge6%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B6%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M366.3002%2C-94.3048C362.1682%2C-85.636%20357.7687%2C-76.406%20353.6206%2C-67.7034%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22356.7049%2C-66.0396%20349.2426%2C-58.5186%20350.386%2C-69.0516%20356.7049%2C-66.0396%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%207%20--%3E%0A%3Cg%20id%3D%22node8%22%20class%3D%22node%22%3E%0A%3Ctitle%3E7%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22518.3044%2C-58.6014%20406.2312%2C-58.6014%20406.2312%2C.2014%20518.3044%2C.2014%20518.3044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22462.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.38%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22462.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20149%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22462.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B111%2C%2038%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%205%26%2345%3B%26gt%3B7%20--%3E%0A%3Cg%20id%3D%22edge7%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E5-%26gt%3B7%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M412.8692%2C-94.3048C419.7266%2C-85.2671%20427.0468%2C-75.6195%20433.8939%2C-66.5954%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22436.7658%2C-68.6007%20440.0221%2C-58.5186%20431.1893%2C-64.3694%20436.7658%2C-68.6007%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%209%20--%3E%0A%3Cg%20id%3D%22node10%22%20class%3D%22node%22%3E%0A%3Ctitle%3E9%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22699.677%2C-169.7003%20540.8586%2C-169.7003%20540.8586%2C-94.2997%20699.677%2C-94.2997%20699.677%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Etotal%20eve%20charge%20%26lt%3B%3D%2015.96%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.359%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20158%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22620.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B37%2C%20121%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B9%20--%3E%0A%3Cg%20id%3D%22edge9%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B9%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M620.2678%2C-205.5528C620.2678%2C-197.22%20620.2678%2C-188.2845%20620.2678%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22623.7679%2C-179.6084%20620.2678%2C-169.6085%20616.7679%2C-179.6085%20623.7679%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2012%20--%3E%0A%3Cg%20id%3D%22node13%22%20class%3D%22node%22%3E%0A%3Ctitle%3E12%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22903.42%2C-169.7003%20749.1156%2C-169.7003%20749.1156%2C-94.2997%20903.42%2C-94.2997%20903.42%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Einternational%20plan%20%26lt%3B%3D%200.5%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.201%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2053%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B47%2C%206%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%208%26%2345%3B%26gt%3B12%20--%3E%0A%3Cg%20id%3D%22edge12%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E8-%26gt%3B12%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M690.01%2C-205.5528C708.5507%2C-195.5444%20728.702%2C-184.6666%20747.6801%2C-174.4221%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22749.4602%2C-177.4386%20756.5974%2C-169.6085%20746.1351%2C-171.2788%20749.4602%2C-177.4386%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2010%20--%3E%0A%3Cg%20id%3D%22node11%22%20class%3D%22node%22%3E%0A%3Ctitle%3E10%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22641.8044%2C-58.6014%20536.7312%2C-58.6014%20536.7312%2C.2014%20641.8044%2C.2014%20641.8044%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22589.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.492%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22589.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2057%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22589.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B32%2C%2025%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%209%26%2345%3B%26gt%3B10%20--%3E%0A%3Cg%20id%3D%22edge10%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E9-%26gt%3B10%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M608.9006%2C-94.3048C606.342%2C-85.8204%20603.6214%2C-76.7985%20601.0465%2C-68.2596%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22604.3472%2C-67.0823%20598.109%2C-58.5186%20597.6453%2C-69.1033%20604.3472%2C-67.0823%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2011%20--%3E%0A%3Cg%20id%3D%22node12%22%20class%3D%22node%22%3E%0A%3Ctitle%3E11%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22758.3045%2C-58.6014%20660.2311%2C-58.6014%20660.2311%2C.2014%20758.3045%2C.2014%20758.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22709.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.094%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22709.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20101%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22709.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B5%2C%2096%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%209%26%2345%3B%26gt%3B11%20--%3E%0A%3Cg%20id%3D%22edge11%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E9-%26gt%3B11%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M652.9027%2C-94.3048C660.8071%2C-85.1748%20669.2502%2C-75.4225%20677.1314%2C-66.3193%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22679.9856%2C-68.3698%20683.8849%2C-58.5186%20674.6934%2C-63.788%20679.9856%2C-68.3698%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2013%20--%3E%0A%3Cg%20id%3D%22node14%22%20class%3D%22node%22%3E%0A%3Ctitle%3E13%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22875.3045%2C-58.6014%20777.2311%2C-58.6014%20777.2311%2C.2014%20875.3045%2C.2014%20875.3045%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.043%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2045%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22826.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B44%2C%201%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B13%20--%3E%0A%3Cg%20id%3D%22edge13%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B13%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M826.2678%2C-94.3048C826.2678%2C-85.9126%20826.2678%2C-76.9946%20826.2678%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22829.7679%2C-68.5186%20826.2678%2C-58.5186%20822.7679%2C-68.5187%20829.7679%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2014%20--%3E%0A%3Cg%20id%3D%22node15%22%20class%3D%22node%22%3E%0A%3Ctitle%3E14%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22984.8046%2C-58.6014%20893.731%2C-58.6014%20893.731%2C.2014%20984.8046%2C.2014%20984.8046%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22939.2678%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Egini%20%3D%200.469%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22939.2678%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%208%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22939.2678%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%20%5B3%2C%205%5D%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2012%26%2345%3B%26gt%3B14%20--%3E%0A%3Cg%20id%3D%22edge14%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E12-%26gt%3B14%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M867.7032%2C-94.3048C878.0431%2C-84.8982%20889.1093%2C-74.8309%20899.3731%2C-65.4936%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22901.9983%2C-67.837%20907.0401%2C-58.5186%20897.2878%2C-62.659%20901.9983%2C-67.837%22%2F%3E%0A%3C%2Fg%3E%0A%3C%2Fg%3E%0A%3C%2Fsvg%3E)

In [ ]:
features = dict(zip(range(len(X.columns)), X.columns))

# Важность признаков
importances = best_tree.feature_importances_

indices = np.argsort(importances)[::-1]
# Plot the feature importancies of the forest
num_to_plot = max(10, len(X.columns))
feature_indices = [ind for ind in indices[:num_to_plot]]

# Print the feature ranking
print("Feature ranking:")

for f in range(num_to_plot):
    print(f+1, features[feature_indices[f]], importances[indices[f]])

plt.figure(figsize=(15,5))
plt.title("Feature importances")
bars = plt.bar(range(num_to_plot), 
               importances[indices[:num_to_plot]],
               color=([str(i/float(num_to_plot+1)) for i in range(num_to_plot)]),
               align="center")
ticks = plt.xticks(range(num_to_plot), 
                   feature_indices)
plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(features[i]) for i in feature_indices]);

Можно предположить, что ночніе действия не влияют на таргет. Также чт total day charge and customer service calls могут быть самыми важными фичами. Точнее о важности ффич скажет лес.

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
f1_score(y_pred, y_test)

In [ ]:
params = {"n_estimators":[50, 100, 150, 200, 300, 400]}
forest_grid_estimators = GridSearchCV(forest, params, cv=kf, scoring='f1')
forest_grid_estimators.fit(X, y)
forest_grid_estimators.best_params_

In [ ]:
params = {"max_depth":np.arange(2,21)}
forest = RandomForestClassifier(n_estimators = 300)
forest_grid_depth = GridSearchCV(tree, params, cv=kf, scoring='f1')
forest_grid_depth.fit(X, y)
forest_grid_depth.best_params_

In [ ]:
params = {"min_samples_split":np.arange(2,21)}
forest = RandomForestClassifier(n_estimators = 300, max_depth = 10)
forest_grid_samples_split = GridSearchCV(tree, params, cv=kf, scoring='f1')
forest_grid_samples_split.fit(X, y)
forest_grid_samples_split.best_params_

In [ ]:
params = {"min_samples_leaf":np.arange(2,21)}
forest = RandomForestClassifier(n_estimators = 300, max_depth= 10, min_samples_split = 8)
forest_grid_samples_leaf = GridSearchCV(tree, params, cv=kf, scoring='f1')
forest_grid_samples_leaf.fit(X, y)
forest_grid_samples_leaf.best_params_

In [ ]:
params = {"max_features":np.arange(2,18)}
forest = RandomForestClassifier(n_estimators = 300, max_depth = 10, min_samples_split = 8, min_samples_leaf = 2)
forest_grid_features = GridSearchCV(tree, params, cv=kf, scoring='f1')
forest_grid_features.fit(X, y)
forest_grid_features.best_params_

In [ ]:
result_estimators = pd.DataFrame(forest_grid_estimators.cv_results_)
result_depth = pd.DataFrame(forest_grid_depth.cv_results_)
result_samples_split = pd.DataFrame(forest_grid_samples_split.cv_results_)
result_samples_leaf = pd.DataFrame(forest_grid_samples_leaf.cv_results_)
result_features = pd.DataFrame(forest_grid_features.cv_results_)

In [ ]:
import matplotlib.pyplot as plt
fig, ((ax11,ax12, ax13),(ax21,ax22, ax23)) = plt.subplots(2,3, figsize = (15,10))

ax11.set_xlabel("n_estimators")
ax11.set_ylabel("f1 score")
ax11.plot(result_estimators["param_n_estimators"], result_estimators["mean_test_score"])

ax12.set_xlabel("max_depth")
ax12.plot(results_depth["param_max_depth"], results_depth["mean_test_score"])

ax13.set_xlabel("min_samples_split")
ax13.plot(result_samples_split["param_min_samples_split"], result_samples_split["mean_test_score"])

ax21.set_xlabel("min_samples_leaf")
ax21.set_ylabel("f1 score")
ax21.plot(result_samples_leaf["param_min_samples_leaf"], result_samples_leaf["mean_test_score"])

ax22.set_xlabel("max_features")
ax22.plot(result_features["param_max_features"], result_features["mean_test_score"])

In [ ]:
best_forest = RandomForestClassifier(n_estimators = 300, max_depth = 10, min_samples_split = 8, min_samples_leaf = 2, max_features = 16)

In [ ]:
features = dict(zip(range(len(X.columns)), X.columns))

# Важность признаков
importances = best_tree.feature_importances_

indices = np.argsort(importances)[::-1]
# Plot the feature importancies of the forest
num_to_plot = max(10, len(X.columns))
feature_indices = [ind for ind in indices[:num_to_plot]]

# Print the feature ranking
print("Feature ranking:")

for f in range(num_to_plot):
    print(f+1, features[feature_indices[f]], importances[indices[f]])

plt.figure(figsize=(15,5))
plt.title("Feature importances")
bars = plt.bar(range(num_to_plot), 
               importances[indices[:num_to_plot]],
               color=([str(i/float(num_to_plot+1)) for i in range(num_to_plot)]),
               align="center")
ticks = plt.xticks(range(num_to_plot), 
                   feature_indices)
plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(features[i]) for i in feature_indices]);

Как и предпологалось ранее: ночные события не важны и главными фичами являются total day charge and customer service calls

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
best_knn = KNeighborsClassifier(weights='distance', p = 1.49, n_neighbors=3)

In [ ]:
from sklearn.model_selection import cross_val_score
print("KNN: ", cross_val_score(best_knn, X, y, cv = kf, scoring = "f1").mean())
print("Tree: ", cross_val_score(best_tree, X, y, cv = kf, scoring = "f1").mean())
print("Forest: ", cross_val_score(best_forest, X, y, cv = kf, scoring = "f1").mean())

Дерево отработало лучше чем knn, но лучше всех отработал случайный лес. Как и предпологалось ибо случайный лес как модель сильнее одного дерева (без бустинга) однако оно плохо интерпретируемо.
